In [1]:
import numpy as np
import pandas as pd
import os
import shutil
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import cv2
import numpy as np

In [2]:

def resize_image(src_image, size=(32,32), bg_color=[79, 69, 54]): 
    org_size = src_image.shape
    l = 0
    t = 0
    r = 0
    b = 0
    cr_l = 0
    cr_t = 0
    cr_r = org_size[1]
    cr_b = org_size[0]
    if org_size[1] < 32:
        l = int(np.ceil((32 - org_size[1])/2))
        r = int(np.floor((32 - org_size[1])/2))
    elif org_size[1] > 32:
        cr_l = int(np.ceil((org_size[1] - 32)/2))
        cr_r = int(org_size[1] - np.floor((org_size[1] - 32)/2))
    if org_size[0] > 32:
        cr_t = int(np.ceil((org_size[0] - 32)/2))
        cr_b = int(org_size[0] - np.floor((org_size[0] - 32)/2))
    elif org_size[0] < 32:
        t = int(np.ceil((32 - org_size[0])/2))
        b = int(np.floor((32 - org_size[0])/2))

    src_image = src_image[cr_t:cr_b, cr_l:cr_r]
    new_image = cv2.copyMakeBorder(src_image, t,b,l,r, borderType=cv2.BORDER_CONSTANT, value=bg_color)
  
    return new_image


In [3]:
classes = ["0", "1"]


train_folder = 'train_images'


for root, subfolders, files in os.walk(train_folder):
    for file_name in files:
        file_path = os.path.join(root, file_name)
        image = cv2.imread(file_path)
        resized_image = resize_image(image)
        cv2.imwrite(file_path, resized_image)


In [4]:
def load_dataset(data_path):

    transformation = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    # Load all of the images, transforming them
    full_dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        transform=transformation
    )
    
    

    train_loader = torch.utils.data.DataLoader(
        full_dataset,
        batch_size=770,
        num_workers=0,
        shuffle=False
    )
    
    
    return train_loader

train_folder = 'train_images'

# Get the iterative dataloaders for test and training data
train_loader = load_dataset(train_folder)
batch_size = train_loader.batch_size

In [ ]:
class Net(nn.Module):
    
    
    
    def __init__(self, num_classes=2):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=5)
        
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=5)
        
        self.pool = nn.MaxPool2d(2, 2)
       
        self.fc1 = nn.Linear(5 * 5 * 24, out_features=120)

        self.fc2 = nn.Linear(120, out_features=num_classes)

    def forward(self, x):
      
        x = F.relu(self.pool(self.conv1(x))) 
        x = F.relu(self.pool(self.conv2(x)))  
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return torch.log_softmax(x, dim=1)
    
device = "cpu"
model = Net(num_classes=len(classes)).to(device)

print(model)

Net(
  (conv1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(12, 24, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=600, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=2, bias=True)
)


In [6]:
def train(model, device, train_loader, optimizer, epoch):
    # Set the model to training mode
    model.train()
    train_loss = 0
    print("Epoch:", epoch)
    # Process the images in batches
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_criteria(output, target)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        print('\tTraining batch {} Loss: {:.6f}'.format(batch_idx + 1, loss.item()))

    avg_loss = train_loss / (batch_idx+1)
    print('Training set: Average loss: {:.6f}'.format(avg_loss))
    return avg_loss

In [10]:
optimizer = optim.Adam(model.parameters(), lr=0.01)

loss_criteria = nn.CrossEntropyLoss()

epoch_nums = [0]
training_loss = [999]
epochs = 50
print('Training on', device)
for epoch in range(1, epochs + 1):
        train_loss = train(model, device, train_loader, optimizer, epoch)
        epoch_nums.append(epoch)
        training_loss.append(train_loss) 
        if epoch > 10 and training_loss[-2] - train_loss < 0.01:
                print(training_loss[-1], train_loss)
                break
        

Training on cpu
Epoch: 1
	Training batch 1 Loss: 0.634321
Training set: Average loss: 0.634321
Epoch: 2
	Training batch 1 Loss: 0.503531
Training set: Average loss: 0.503531
Epoch: 3
	Training batch 1 Loss: 0.488922
Training set: Average loss: 0.488922
Epoch: 4
	Training batch 1 Loss: 0.463278
Training set: Average loss: 0.463278
Epoch: 5
	Training batch 1 Loss: 0.465249
Training set: Average loss: 0.465249
Epoch: 6
	Training batch 1 Loss: 0.458334
Training set: Average loss: 0.458334
Epoch: 7
	Training batch 1 Loss: 0.424054
Training set: Average loss: 0.424054
Epoch: 8
	Training batch 1 Loss: 0.427353
Training set: Average loss: 0.427353
Epoch: 9
	Training batch 1 Loss: 0.381081
Training set: Average loss: 0.381081
Epoch: 10
	Training batch 1 Loss: 0.388016
Training set: Average loss: 0.388016
Epoch: 11
	Training batch 1 Loss: 0.337151
Training set: Average loss: 0.337151
Epoch: 12
	Training batch 1 Loss: 0.319826
Training set: Average loss: 0.319826
Epoch: 13
	Training batch 1 Loss:

In [ ]:
torch.save(model.state_dict(), 'basket_cl2.pt')